# Data Gathering

### programmatically downloaded using the Requests library

In [2]:
#import libraries
import requests
import pandas as pd
import json
import datetime as dt

In [4]:
#store file to file handler
r = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')
code = r.status_code
print(code)
#save downloaded file to local folder
open('image-predictions.tsv', 'wb').write(r.content)



200


335079

### Download tweet ID, retweet count, and favorite count using Tweepy API

In [ ]:
#read twitte IDs

#read the twitter-archive-enhanced.csv file in to dataframe
df_1 = pd.read_csv('twitter-archive-enhanced.csv')
df_1
#convert the first column of the datafram(tweet id) to a list
IDlist = df_1['tweet_id'].tolist()
IDlist

In [ ]:
#pull twitter status json from twitter API and store them into a list
import tweepy

# key and token omitted for privacy
consumer_key = '*'
consumer_secret = '*'
access_token = '*'
access_secret = '*'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit  = True, wait_on_rate_limit_notify = True)

tweet_list = []

for tid in IDlist:
    try:
        tweet = api.get_status(tid, tweet_mode='extended')
        tweet_list.append(tweet._json)
    except:
        print('tweet ' + str(tid) + "doesn't" exist')



In [5]:
#check if all tweet status has been downloaded
len(tweet_list)

2348

In [ ]:
# write json to txt file

# type(tweet_json)
with open('tweet_json.txt', 'w') as outfile:
    for tweet_json in tweet_list:
        json.dump(tweet_json, outfile)
        outfile.write('\n') #add a newline character at the end of each json

In [5]:
# read json from text

tweet_df_raw = pd.read_json('tweet_json.txt', lines = 'True')
tweet_df = tweet_df_raw[['id', 'retweet_count', 'favorite_count']]

tweet_df

,id,retweet_count,favorite_count
0,892420643555336193,8758,39245
1,892177421306343426,6422,33609
2,891815181378084864,4265,25317
3,891689557279858688,8849,42611
4,891327558926688256,9631,40774
5,891087950875897856,3206,20444
6,890971913173991426,2124,11987
7,890729181411237888,19374,66310
8,890609185150312448,4362,28058
9,890240255349198849,7604,32303


# Data Assessing

In [6]:
#load 3 dataframes from different sources
archive_df = pd.read_csv('twitter-archive-enhanced.csv')
image_df = pd.read_csv('image-predictions.tsv', sep='\t')
status_df = tweet_df.copy()
status_df.rename(columns = {"id": "tweet_id"}, inplace = True)

In [7]:
status_df

,tweet_id,retweet_count,favorite_count
0,892420643555336193,8758,39245
1,892177421306343426,6422,33609
2,891815181378084864,4265,25317
3,891689557279858688,8849,42611
4,891327558926688256,9631,40774
5,891087950875897856,3206,20444
6,890971913173991426,2124,11987
7,890729181411237888,19374,66310
8,890609185150312448,4362,28058
9,890240255349198849,7604,32303


In [8]:
archive_df.info()
archive_df.sample()
type(archive_df.timestamp)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

pandas.core.series.Series

In [9]:
archive_df.rating_denominator.value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

issues in archive dataframe
1. Quality issues:
    - Retweets: some of the tweets in this dataframe are retweet, as mentioned in the project detail. These retweet are not supposed to be included in the analysis
    - Unnecessary information: text, sources are not needed for analysis. Retweeted_status_id and retweeted_status_user_id, and retweetd_status_timestamp are not needed after data cleaning procedure. 
    - The rating_numerator and rating_denominator can be combined into one column in decimal form.
    - Wrong data type for tweet id. Since no calculations will be applied on tweet ID, the tweet ID needs to be str instead of int64.
    - The timestamp column has wrong data type. 
2. Tidiess issues
    - Dog stages are not in one column, instead, they are divided into 4.
    - Date and time should be two variables for the purpose of analysis. 

In [10]:
image_df.info()
image_df.p1.value_counts()
#image_df.sample()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


golden_retriever             150
Labrador_retriever           100
Pembroke                      89
Chihuahua                     83
pug                           57
chow                          44
Samoyed                       43
toy_poodle                    39
Pomeranian                    38
malamute                      30
cocker_spaniel                30
French_bulldog                26
miniature_pinscher            23
Chesapeake_Bay_retriever      23
seat_belt                     22
German_shepherd               20
Siberian_husky                20
Staffordshire_bullterrier     20
Cardigan                      19
web_site                      19
beagle                        18
teddy                         18
Eskimo_dog                    18
Maltese_dog                   18
Shetland_sheepdog             18
Rottweiler                    17
Shih-Tzu                      17
Lakeland_terrier              17
kuvasz                        16
Italian_greyhound             16
          

issues in image dataframe
1. Quality issues:
    - Non-descriptive column headers: p1, p1_conf, p1_dog, p2, p2_dog, p3, p3_conf, p3_dog etc. 
    - Some of the dog breeds has first letter capitalized and some are not. 
    - Some of the dog breeds are misspelled (ie. 19 of the breeds is website) or intended misspelled (ie. cheeseburger) 
    
2. Tidiess issues

In [11]:
status_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2348 entries, 0 to 2347
Data columns (total 3 columns):
tweet_id          2348 non-null int64
retweet_count     2348 non-null int64
favorite_count    2348 non-null int64
dtypes: int64(3)
memory usage: 73.4 KB


issues in status dataframe
1. Quality issues:
    - wrong data type for tweet ID
2. Tidiess issues

### Summary
Quality issues:
    1. Retweets: some of the tweets in this dataframe are retweet. as mentioned in the project detail, These retweets are not supposed to be included in the analysis.
    2. Unnecessary information: text, sources are not needed for analysis. Retweeted_status_id and retweeted_status_user_id, and retweetd_status_timestamp are not needed after data cleaning procedure. 
    3. Wrong data type for tweet id. Since no calculations will be applied on tweet ID, the tweet ID needs to be str instead of int64.
    4. The timestamp column has wrong data type. 
    5. Non-descriptive column headers: p1, p1_conf, p1_dog, p2, p2_dog, p3, p3_conf, p3_dog etc. 
    6. Some of the dog breeds has first letter capitalized and some are not. 
    7. Many ratings' denominator are not 10, even the numerator which greater than 10 is the feature of this twitter account, keep the denominator same is vital to later analysis. 
    8. Missing data: the image prediction part dataframe has 2075 rows while the status df has 2348 and the archive has 2356. Observations that has missing image predictions are considered as missing data.

Tidyness issues:
    1. Dog stages are not in one column, instead, they are divided into 4.
    2. One observation unit does not form a table. At least, retweet_count and favorite_count needs to be part of the archive dataframe to form a complete observation unit. 
    *3. archive dataframe, time and dtae should be two variables instead of 1.

# Data Cleaning

Issue #1 : retweets

#### Define:
Remove all tweet rows that has nonnull value in retweeted_status_id column in archive dataframe.

#### Code

In [12]:
# create a copy
archive_df_clean = archive_df.copy()

# only keep tweet whose retweeted_status_id is NAn
archive_df_clean = archive_df_clean[archive_df_clean['retweeted_status_id'].isnull()]

#### Test

In [13]:
archive_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2175 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2175 non-null object
source                        2175 non-null object
text                          2175 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 2117 non-null object
rating_numerator              2175 non-null int64
rating_denominator            2175 non-null int64
name                          2175 non-null object
doggo                         2175 non-null object
floofer                       2175 non-null object
pupper                        2175 non-null object
puppo                         2175 non-null object
dtypes: float64(4), int64(3), object(1

Issue #2: Unnecessary information
#### Define:
Remove columns: expanded_urls, in_reply_to_status_id, in_reply_to_user_id, text, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp, source. 

#### Code

In [14]:
# Drop unwanted columns
archive_df_clean = archive_df_clean.drop(columns=['expanded_urls', 'in_reply_to_status_id', 'in_reply_to_user_id', 'text', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 'source'])

#### Test

In [15]:
list(archive_df_clean)

['tweet_id',
 'timestamp',
 'rating_numerator',
 'rating_denominator',
 'name',
 'doggo',
 'floofer',
 'pupper',
 'puppo']

Issue #3: Wrong datatype for tweet ID
#### Define:
Change data type of Tweet ID to string

#### Code

In [16]:
# change the datatype of tweet_id
archive_df_clean['tweet_id'] = archive_df_clean['tweet_id'].astype(str)
# change the datatype of tweet_id column in status_df as well
status_df['tweet_id'] = status_df['tweet_id'].astype(str)



#### Test

In [17]:
archive_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 9 columns):
tweet_id              2175 non-null object
timestamp             2175 non-null object
rating_numerator      2175 non-null int64
rating_denominator    2175 non-null int64
name                  2175 non-null object
doggo                 2175 non-null object
floofer               2175 non-null object
pupper                2175 non-null object
puppo                 2175 non-null object
dtypes: int64(2), object(7)
memory usage: 169.9+ KB


Issue #4: Wrong datatype for time stamp column
#### Define:
Remove columns: change the timestamp column to 

#### Code

In [18]:
#analysis the timestamp column
type(archive_df_clean['timestamp'])

archive_df_clean.head(100)

,tweet_id,timestamp,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,2017-08-01 16:23:56 +0000,13,10,Phineas,None,None,None,None
1,892177421306343426,2017-08-01 00:17:27 +0000,13,10,Tilly,None,None,None,None
2,891815181378084864,2017-07-31 00:18:03 +0000,12,10,Archie,None,None,None,None
3,891689557279858688,2017-07-30 15:58:51 +0000,13,10,Darla,None,None,None,None
4,891327558926688256,2017-07-29 16:00:24 +0000,12,10,Franklin,None,None,None,None
5,891087950875897856,2017-07-29 00:08:17 +0000,13,10,None,None,None,None,None
6,890971913173991426,2017-07-28 16:27:12 +0000,13,10,Jax,None,None,None,None
7,890729181411237888,2017-07-28 00:22:40 +0000,13,10,None,None,None,None,None
8,890609185150312448,2017-07-27 16:25:51 +0000,13,10,Zoey,None,None,None,None
9,890240255349198849,2017-07-26 15:59:51 +0000,14,10,Cassie,doggo,None,None,None


In [19]:
# single out first 19 digits of the timestamp
archive_df_clean['timestamp'] = archive_df_clean.timestamp.str[:19]

archive_df_clean['timestamp'] = pd.to_datetime(archive_df_clean['timestamp'], format = "%Y-%m-%d %H:%M:%S")

In [21]:
archive_df_clean.head(100)

,tweet_id,timestamp,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,2017-08-01 16:23:56,13,10,Phineas,None,None,None,None
1,892177421306343426,2017-08-01 00:17:27,13,10,Tilly,None,None,None,None
2,891815181378084864,2017-07-31 00:18:03,12,10,Archie,None,None,None,None
3,891689557279858688,2017-07-30 15:58:51,13,10,Darla,None,None,None,None
4,891327558926688256,2017-07-29 16:00:24,12,10,Franklin,None,None,None,None
5,891087950875897856,2017-07-29 00:08:17,13,10,None,None,None,None,None
6,890971913173991426,2017-07-28 16:27:12,13,10,Jax,None,None,None,None
7,890729181411237888,2017-07-28 00:22:40,13,10,None,None,None,None,None
8,890609185150312448,2017-07-27 16:25:51,13,10,Zoey,None,None,None,None
9,890240255349198849,2017-07-26 15:59:51,14,10,Cassie,doggo,None,None,None


#### Test

In [22]:
archive_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 9 columns):
tweet_id              2175 non-null object
timestamp             2175 non-null datetime64[ns]
rating_numerator      2175 non-null int64
rating_denominator    2175 non-null int64
name                  2175 non-null object
doggo                 2175 non-null object
floofer               2175 non-null object
pupper                2175 non-null object
puppo                 2175 non-null object
dtypes: datetime64[ns](1), int64(2), object(6)
memory usage: 169.9+ KB


Issue #5: Non-Descriptive column header in image dataframe
#### Define:
change following column headers:

- p1 -> prediction_1
- p1_conf -> prediction_1_confidence
- p1_dog -> prediction_1_result
- p2 -> prediction_2
- p2_conf -> prediction_2_confidence
- p2_dog -> prediction_2_result
- p3 -> prediction_3
- p3_conf -> prediction_3_confidence
- p3_dog -> prediction_3_result
#### Code

In [23]:
image_df_clean = image_df.copy()
image_df_clean.rename(columns={'p1': 'prediction_1', 'p1_conf': 'prediction_1_confidence', 'p1_dog': 'prediction_1_result'}, inplace=True)
image_df_clean.rename(columns={'p2': 'prediction_2', 'p2_conf': 'prediction_2_confidence', 'p2_dog': 'prediction_2_result'}, inplace=True)
image_df_clean.rename(columns={'p3': 'prediction_3', 'p3_conf': 'prediction_3_confidence', 'p3_dog': 'prediction_3_result'}, inplace=True)

#### Test

In [24]:
image_df_clean.sample(100)

,tweet_id,jpg_url,img_num,prediction_1,prediction_1_confidence,prediction_1_result,prediction_2,prediction_2_confidence,prediction_2_result,prediction_3,prediction_3_confidence,prediction_3_result
525,676606785097199616,https://pbs.twimg.com/media/CWPKSGpWcAQN6mw.jpg,1,Loafer,0.202999,False,doormat,0.200411,False,malinois,0.142300,True
424,674063288070742018,https://pbs.twimg.com/media/CVrA-rIWEAANxwQ.jpg,1,ostrich,0.661176,False,bearskin,0.214879,False,swab,0.064456,False
622,680497766108381184,https://pbs.twimg.com/media/CXGdG0aWcAEbOO1.jpg,1,Chihuahua,0.538354,True,muzzle,0.084289,False,ski_mask,0.076690,False
22,666337882303524864,https://pbs.twimg.com/media/CT9OwFIWEAMuRje.jpg,1,ox,0.416669,False,Newfoundland,0.278407,True,groenendael,0.102643,True
1477,780543529827336192,https://pbs.twimg.com/media/CtUMLzRXgAAbZK5.jpg,1,golden_retriever,0.628312,True,Labrador_retriever,0.317365,True,Tibetan_mastiff,0.012260,True
1296,752173152931807232,https://pbs.twimg.com/media/CnBBfNuWcAAkOgO.jpg,1,Labrador_retriever,0.527659,True,German_shepherd,0.174765,True,Chihuahua,0.045525,True
735,687096057537363968,https://pbs.twimg.com/media/CYkON6CVAAAPXAc.jpg,1,Labrador_retriever,0.417107,True,Chesapeake_Bay_retriever,0.341730,True,German_short-haired_pointer,0.177702,True
696,684481074559381504,https://pbs.twimg.com/media/CX_D6AJWwAAnBIw.jpg,1,Chihuahua,0.937810,True,Pomeranian,0.020307,True,polecat,0.017357,False
414,674008982932058114,https://pbs.twimg.com/media/CVqPkVoU4AAkXA7.jpg,1,jigsaw_puzzle,0.970810,False,prayer_rug,0.011048,False,quill,0.008432,False
2005,877556246731214848,https://pbs.twimg.com/media/DC20wEcW0AAf59m.jpg,1,basset,0.995368,True,Welsh_springer_spaniel,0.001936,True,bathtub,0.000468,False


Issue #6: the first letter of all dog names are not all capitalized.
#### Define:
capitalize the first letter of all dog breeds

#### Code

In [25]:
image_df_clean.prediction_1 = image_df_clean.prediction_1.str.title()
image_df_clean.prediction_2 = image_df_clean.prediction_2.str.title()
image_df_clean.prediction_3 = image_df_clean.prediction_3.str.title()


#### Test

In [26]:
image_df_clean.sample(100)

,tweet_id,jpg_url,img_num,prediction_1,prediction_1_confidence,prediction_1_result,prediction_2,prediction_2_confidence,prediction_2_result,prediction_3,prediction_3_confidence,prediction_3_result
1289,751251247299190784,https://pbs.twimg.com/ext_tw_video_thumb/75125...,1,Walker_Hound,0.178852,True,German_Short-Haired_Pointer,0.115752,True,English_Foxhound,0.113796,True
754,688211956440801280,https://pbs.twimg.com/ext_tw_video_thumb/68821...,1,Bannister,0.369449,False,Four-Poster,0.105307,False,Shoji,0.098767,False
1001,708738143638450176,https://pbs.twimg.com/media/CdXxlFPWwAABaOv.jpg,1,Pomeranian,0.933457,True,Samoyed,0.057221,True,West_Highland_White_Terrier,0.000904,True
884,698953797952008193,https://pbs.twimg.com/media/CbMuxV5WEAAIBjy.jpg,1,Italian_Greyhound,0.382378,True,Redbone,0.102255,True,Shower_Cap,0.076834,False
520,676533798876651520,https://pbs.twimg.com/media/CWOH4s9U8AEtkmQ.jpg,1,Chow,0.265274,True,Ice_Bear,0.167614,False,Fur_Coat,0.117506,False
1432,773308824254029826,https://pbs.twimg.com/media/CrtYRMEWIAAUkCl.jpg,1,Shopping_Cart,0.572349,False,Labrador_Retriever,0.151406,True,Shopping_Basket,0.107102,False
1797,831552930092285952,https://pbs.twimg.com/media/C4pE-I0WQAABveu.jpg,1,Chihuahua,0.257415,True,Pembroke,0.161442,True,French_Bulldog,0.092143,True
976,707021089608753152,https://pbs.twimg.com/media/Cc_XtkRW8AEE7Fn.jpg,2,Cocker_Spaniel,0.559658,True,Golden_Retriever,0.314673,True,Pekinese,0.066672,True
733,686749460672679938,https://pbs.twimg.com/media/CYfS75fWAAAllde.jpg,1,Cheeseburger,0.643808,False,Hotdog,0.201378,False,Bagel,0.063880,False
145,668625577880875008,https://pbs.twimg.com/media/CUdvambWoAA007z.jpg,1,Ox,0.071536,False,Groenendael,0.054455,True,Angora,0.045028,False


Issue #7: Many ratings' denominator are not 10
#### Define:
Scale tweets who's denominators more than 10 or less than 10 to 10

#### Code

In [27]:
archive_df_clean.info()
archive_df_clean.rating_denominator.value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 9 columns):
tweet_id              2175 non-null object
timestamp             2175 non-null datetime64[ns]
rating_numerator      2175 non-null int64
rating_denominator    2175 non-null int64
name                  2175 non-null object
doggo                 2175 non-null object
floofer               2175 non-null object
pupper                2175 non-null object
puppo                 2175 non-null object
dtypes: datetime64[ns](1), int64(2), object(6)
memory usage: 169.9+ KB


10     2153
50        3
80        2
11        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

In [36]:
for index, row in archive_df_clean.iterrows():
    if row['rating_denominator'] == 0:
        new_denominator = 10
        archive_df_clean.set_value(index, 'rating_denominator', new_denominator)
    elif row['rating_denominator'] != 10:
        print(row['tweet_id'])
        new_denominator = 10
        new_numerator = round((row['rating_numerator']/row['rating_denominator'])*10, 1)
        print(row['rating_denominator'])
        print(new_denominator)
        print(row['rating_numerator'])
        print(new_numerator)
        archive_df_clean.set_value(index, 'rating_denominator', new_denominator)
        archive_df_clean.set_value(index, 'rating_numerator', new_numerator)
#     else:
#         new_denominator = rating_denominator
#         new_numerator = rating_numerator
    

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


#### Test

In [37]:
archive_df_clean.rating_denominator.value_counts()

10    2175
Name: rating_denominator, dtype: int64

Issue #8: Missing data
#### Define:
inner join 3 dataframe and make sure a tweet record has information in all 3 dataframe. 

#### Code#

In [ ]:
archive_df_clean.info()
image_df_clean.info()
status_df.info()
archive_df_clean_1 = archive_df_clean.copy()
type(archive_df_clean)


In [ ]:
# status_df_clean = status_df.copy()

# short_ID_List = image_df_clean['tweet_id']
# short_ID_List = image_df_clean.astype(str)

# archive_df_clean = archive_df_clean[archive_df_clean['tweet_id'].isin(short_ID_List)]
# status_df_clean = status_df_clean[status_df_clean["tweet_id"].isin(short_ID_List)]






In [ ]:


archive_df_clean.info()
status_df_clean.info()
#type(short_ID_List['tweet_id'][1])
#type(status_df_clean["tweet_id"][1])
#type(archive_df_clean['tweet_id'][1])

tidyness issue #2: dog stages not in on column
#### Define
combine doggo, floofer, pupper and puppo columns into stage column

#### Code



In [ ]:
#archive_df_clean['stage'] = archive_df_clean['doggo'] + archive_df_clean['floofer'] +  archive_df_clean['pupper'] + archive_df_clean['puppo'] 

for index, row in archive_df_clean.iterrows():
    if row["doggo"]  == "doggo" or row["floofer"] == "floofer" or row["pupper"] == "pupper" or row["puppo"] == ["puppo"]:
        archive_df_clean
        

#### Test

In [38]:
archive_df_clean.sample(100)

,tweet_id,timestamp,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
724,782747134529531904,2016-10-03 01:00:34,11,10,Deacon,doggo,None,None,None
62,880095782870896641,2017-06-28 16:09:20,11,10,None,None,None,None,None
83,876537666061221889,2017-06-18 20:30:39,14,10,None,None,None,None,None
2210,668620235289837568,2015-11-23 02:41:01,10,10,Kallie,None,None,None,None
1731,679844490799091713,2015-12-24 02:02:12,10,10,Willie,None,None,None,None
45,883482846933004288,2017-07-08 00:28:19,5,10,Bella,None,None,None,None
1133,728751179681943552,2016-05-07 00:59:55,10,10,Flurpson,None,None,None,None
1322,706291001778950144,2016-03-06 01:31:11,11,10,None,None,None,None,None
1854,675534494439489536,2015-12-12 04:35:48,11,10,a,None,None,None,None
1464,694356675654983680,2016-02-02 03:08:26,10,10,None,None,None,pupper,None


# Analysis and Visualization

In [ ]:
import matplotlib as mp



# reference

1. multiple json in txt line by line: https://stackoverflow.com/questions/17055117/python-json-dump-append-to-txt-with-each-variable-on-new-line
2. load json from txt line by line: https://stackoverflow.com/questions/21533894/how-to-read-line-delimited-json-from-large-file-line-by-line
3. create dataframe from json file: https://stackoverflow.com/questions/47925828/how-to-create-a-pandas-dataframe-using-tweepy
4. keep only rows who has NaN in a certain column: https://stackoverflow.com/questions/25430995/keeping-nan-values-and-dropping-nonmissing-values
5. str to datetime: https://stackoverflow.com/questions/17134716/convert-dataframe-column-type-from-string-to-datetime
6. iterate rows to change value https://stackoverflow.com/questions/23330654/update-a-dataframe-in-pandas-while-iterating-row-by-row
7. use isin
8. combine to different columns into 1: 